# Full Search

## Load Bible Text

In [34]:
import pandas as pd
import numpy as np

types = {
    'book': 'string',
    'chap': np.int32,
    'vers': np.int32,
    'text': 'string'
}
unv = pd.read_csv(
    'https://bible.fhl.net/public/dnstrunv.tgz',
    sep='#',
    compression='gzip',
    header=None,
    usecols=[1, 2, 3, 4],
    names=list(types.keys()))

In [35]:
unv[unv.book.notna() & (unv.book.str.startswith('Deut')) & (unv.chap == 28.0) & (unv.vers > 40.0)].sort_values('vers')[:10]

,book,chap,vers,text
6366,Deut,28.0,41.0,你生兒養女，卻不算是你的，因為必被擄去。
7045,Deut,28.0,42.0,你所有的樹木和你地裡的出產必被蝗蟲所吃。
6318,Deut,28.0,43.0,在你中間寄居的，必漸漸上升，比你高而又高；你必漸漸下降，低而又低。
8929,Deut,28.0,44.0,他必借給你，你卻不能借給他；他必作首，你必作尾。
25236,Deut,28.0,45.0,這一切咒詛必追隨你，趕上你，直到你滅亡；因為你不聽從耶和華─你神的話，不遵守他所吩咐的誡命律例。
6365,Deut,28.0,46.0,這些咒詛必在你和你後裔的身上成為異蹟奇事，直到永遠！
28341,Deut,28.0,47.0,「因為你富有的時候，不歡心樂意地事奉耶和華─你的神，
9896,Deut,28.0,48.0,所以你必在飢餓、乾渴、赤露、缺乏之中事奉耶和華所打發來攻擊你的仇敵。他必把鐵軛加在你的頸項上...
9954,Deut,28.0,49.0,「耶和華要從遠方、地極帶一國的民，如鷹飛來攻擊你。這民的言語，你不懂得。
9918,Deut,28.0,50.0,這民的面貌凶惡，不顧恤年老的，也不恩待年少的。


In [36]:
unv.dropna(inplace=True)
unv = unv.astype(types)

## Tokenize

In [5]:
import jieba
import jieba.posseg as pseg

jieba.enable_paddle()

Paddle enabled successfully......


In [6]:
from hanziconv import HanziConv

unv['s_text'] = unv.text.apply(HanziConv.toSimplified)

In [8]:
unv['s_text_tk'] = unv.s_text.apply(lambda v: pseg.lcut(v, use_paddle=True))

## Search

In [29]:
def highlight_occurances(text: str, keywords: Iterable[str]) -> str:
    for i, kw in enumerate(keywords):
        text = text.replace(kw, highlight(kw, i))
    return text

def highlight(text: str, color_code: int) -> str:
    return f'\x1b[6;30;4{color_code + 1}m{text}\x1b[0m'

test_data = ['first', 'second', 'thrid', 'forth', 'fifth', 'sixth', 'seventh']
print(highlight_occurances(', '.join(test_data), test_data))

first, second, thrid, forth, fifth, sixth, seventh


In [30]:
from typing import Iterable

from search import sentence_similarity

searches = ['挂虑 祈祷', '喜乐 事奉', '求救', '信心 行事']

for search_term in searches:
    print(f'Search for {search_term}:')
    search_tk = pseg.lcut(search_term, use_paddle=True)
    match_scores = {}
    for v, vers_tk in zip(unv.s_text, unv.s_text_tk):
        similarity = sentence_similarity(search_tk, vers_tk)
        match_kw = [kw for kw in similarity.keys() if similarity[kw] > 0]
        vers = highlight_occurances(v, match_kw)
        score = sum(similarity.values())
        match_scores[vers] = score
    for top_match in sorted(match_scores, key=match_scores.get, reverse=True)[:10]:
        print(f'Match: {match_scores[top_match]:7.4f} Verse: {top_match}')
    print()

Search for 挂虑 祈祷:
Match:  2.0000 Verse: 应当一无挂虑，只要凡事借着祷告、祈求，和感谢，将你们所要的告诉神。
Match:  1.1429 Verse: 示门的儿子是暗嫩、林拿、便‧哈南、提伦。以示的儿子是梭黑与便‧梭黑。
Match:  1.1429 Verse: 又到了基列和他停‧合示地，又到了但‧雅安，绕到西顿，
Match:  1.1429 Verse: 俄别‧以东的长子是示玛雅，次子是约萨拔，三子是约亚，四子是沙甲，五子是拿坦业，
Match:  1.1429 Verse: 又派俄别‧以东和他的弟兄六十八人，与耶杜顿的儿子俄别‧以东，并何萨作守门的；
Match:  1.1429 Verse: 为首的是亚萨，其次是撒迦利雅、雅薛、示米拉末、耶歇、玛他提雅、以利押、比拿雅、俄别‧以东、耶利，鼓瑟弹琴；惟有亚萨敲钹，大发响声；
Match:  1.1429 Verse: 于是大卫不肯将耶和华的约柜运进大卫的城，却运到迦特人俄别‧以东的家中。
Match:  1.1429 Verse: 故此，以色列王对便‧哈达的使者说：「你们告诉我主我王说：王头一次差遣人向仆人所要的，仆人都依从；但这次所要的，我不能依从。」使者就去回覆便‧哈达。
Match:  1.1429 Verse: 妇人和处女也有分别。没有出嫁的，是为主的事挂虑，要身体、灵魂都圣洁；已经出嫁的，是为世上的事挂虑，想怎样叫丈夫喜悦。
Match:  1.1429 Verse: 我愿你们无所挂虑。没有娶妻的，是为主的事挂虑，想怎样叫主喜悦。

Search for 喜乐 事奉:
Match:  2.0000 Verse: 你们当乐意事奉耶和华，当来向他歌唱！
Match:  2.0000 Verse: 「以色列家啊，至于你们，主耶和华如此说：从此以后若不听从我，就任凭你们去事奉偶像，只是不可再因你们的供物和偶像亵渎我的圣名。
Match:  2.0000 Verse: 「我儿所罗门哪，你当认识耶和华─你父的神，诚心乐意地事奉他；因为他鉴察众人的心，知道一切心思意念。你若寻求他，他必使你寻见；你若离弃他，他必永远丢弃你。
Match:  2.0000 Verse: 那时，我必使万民用清洁的言语好求告我─耶和华的名，同心合意地事奉我。
Match:  2.0000 Vers